In [2]:
!mkdir images

In [4]:
from bing_image_downloader import downloader

In [7]:
#downloader.download("German Shepherd",limit=100,output_dir='images',adult_filter_off=True)
downloader.download("Bulldog",limit=100,output_dir='images',adult_filter_off=True)

[%] Downloading Images to c:\Users\hp\OneDrive\Documents\full stack datascience\Image_Detection_Streamlit_App_with_DeepLearning\images\Bulldog


[!!]Indexing page: 1

[%] Indexed 35 Images on Page 1.


[%] Downloading Image #1 from http://www.mascotarios.org/wp-content/uploads/2012/03/BulldogIngles.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://2.bp.blogspot.com/-uQGdZGmyng0/T83p7wQV2hI/AAAAAAAABtQ/gwqY3Gx-PYw/s1600/Bulldog.jpeg
[%] File Downloaded !

[%] Downloading Image #3 from http://animalsbreeds.com/wp-content/uploads/2015/01/Bulldog-4.jpg
[%] File Downloaded !

[%] Downloading Image #4 from http://mydoggy.rocks/wp-content/uploads/2014/09/English-bulldog-face.jpg
[%] File Downloaded !

[%] Downloading Image #5 from http://upload.wikimedia.org/wikipedia/commons/e/ef/Female_English_Bulldog.jpg
[%] File Downloaded !

[%] Downloading Image #6 from https://animalsbreeds.com/wp-content/uploads/2015/01/Bulldog-7.jpg
[%] File Downloaded !

[%] Downloading Image #7 from 

In [8]:
import os
import matplotlib.pyplot as plt
 #to read image
from skimage.io import imread
#to resize image to uniform sizes
from skimage.transform import resize

import tensorflow as tf
import numpy as np
import pandas as pd

import keras
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator #, load_img, img_to_array #
from tensorflow.keras.utils import load_img, img_to_array

In [9]:
tf.keras.applications.inception_v3.preprocess_input
from keras.applications.inception_v3 import preprocess_input

In [10]:
base_model = tf.keras.applications.inception_v3.InceptionV3(input_shape=(256,256,3),
                                                           include_top=False
                                                           )


87910968/87910968 [==============================] - 244s 3us/step


In [11]:
for layer in base_model.layers:
    layer.trainable=False

In [12]:
#read on activation functions
X = Flatten()(base_model.output)
X =Dense(units=2, activation = 'sigmoid')(X)

In [13]:
model = Model(base_model.input, X)

#compile model
model.compile(optimizer='adam',loss= keras.losses.binary_crossentropy, metrics=['accuracy'])

#summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 127, 127, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 127, 127, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [14]:
train_datagen = ImageDataGenerator(featurewise_center = True, rotation_range=0.4,
                                  width_shift_range=0.3,
                                  horizontal_flip=True,
                                  preprocessing_function= preprocess_input,
                                  zoom_range=0.4,
                                  shear_range = 0.4)

train_data = train_datagen.flow_from_directory(directory='images',
                                              target_size=(256,256),
                                              batch_size=32)

Found 199 images belonging to 2 classes.


In [15]:
train_data.class_indices

{'Bulldog': 0, 'German Shepherd': 1}

In [16]:
t_img , label = train_data.next()

C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\keras\preprocessing\image.py:1861: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


In [17]:
t_img.shape

(32, 256, 256, 3)

In [ ]:
def plotImages(img_arr, label):
    '''
    input: image array
    output: plot images
    '''
    for idx , img in enumerate(img_arr):
        if idx <= 3:
            plt.figure(figsize(5,5))
            plt.imshow()
            

In [18]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

mc = ModelCheckpoint(filepath='./best_model.h5',
                  monitor = 'accuracy',
                  verbose=1,
                  save_best_only=True)
es= EarlyStopping(monitor='accuracy',
                 min_delta=0.01,
                 patience=5,
                 verbose=1)

cb=[mc,es]

In [19]:
his = model.fit_generator(train_data,
                         steps_per_epoch=10,
                         epochs=30,
                         callbacks=cb)

C:\Users\hp\AppData\Local\Temp\ipykernel_4700\2706618946.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  his = model.fit_generator(train_data,


Epoch 1/30
 7/10 [====================>.........] - ETA: 8s - loss: 0.2696 - accuracy: 0.9196 WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 300 batches). You may need to use the repeat() function when building your dataset.

Epoch 1: accuracy improved from -inf to 0.91960, saving model to .\best_model.h5
10/10 [==============================] - 30s 2s/step - loss: 0.2696 - accuracy: 0.9196


In [20]:
from keras.models import load_model

In [21]:
model = load_model('best_model.h5')

In [22]:
%%writefile img_det.py

import streamlit as st
import pandas as pd
import numpy as np
import tensorflow as tf
from PIL import Image
from io import BytesIO
from tensorflow.keras.utils import load_img, img_to_array
#from keras.preprocessing.image import img_to_array
from keras.applications.inception_v3 import preprocess_input
from keras.models import load_model

model = load_model('best_model.h5')

classes = {0 : 'Bulldog', 1 : 'German Shepherd'}

img_file = st.file_uploader('select an image', type=['jpg','png','jpeg','gif','jfif','heic'])

if img_file is not None :
    img = Image.open(img_file)
    st.image(img,caption='Upload image succesfully')
    
    if st.button('predict'):
        img = img.resize((256,256))
        i = img_to_array(img)
        i = preprocess_input(i)
        input_arr = np.array([i])
        
        y_out = np.argmax(model.predict(input_arr))
        y_out1 = classes[y_out]
        
        #if y_out1 = 0:
        st.write(f'This image is a {y_out1}')
        #else:
            #st.write(f'This image is a {y_out1}')

Overwriting img_det.py
